# 1) En temel ECA bloğu (sabit k ile)

* Önce adaptif k falan yok. Mantığı net görmek için k=3 ile başlıyoruz.

In [ ]:
import torch
import torch.nn as nn

class ECABlock(nn.Module):
    def __init__(self, channels: int, k: int = 3):
        super().__init__()
        if k % 2 == 0:
            k += 1
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.conv1d = nn.Conv1d(1, 1, kernel_size=k, padding=(k - 1)//2, bias=False)
        self.gate = nn.Sigmoid()

# padding=(k-1)//2 seçimi:

# çıktı uzunluğu girişle aynı kalsın diye

# same padding uygulamak için

# ve odd kernel ile simetrik komşuluk sağlamak için

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        s = self.pool(x).squeeze(-1).squeeze(-1)   # (B,C)
        s = s.unsqueeze(1)                         # (B,1,C)
        a = self.conv1d(s)                         # (B,1,C)
        w = self.gate(a).squeeze(1).unsqueeze(-1).unsqueeze(-1)  # (B,C,1,1)
        return x * w

In [2]:
x = torch.randn(2, 64, 56, 56)
eca = ECABlock(64, k=3)
y = eca(x)
print(x.shape, y.shape)

torch.Size([2, 64, 56, 56]) torch.Size([2, 64, 56, 56])


# 2) Adaptif k ekleyelim 

Şimdi k otomatik seçilsin.

In [ ]:
import math

def _make_odd(k: int) -> int:
    return k if (k % 2 == 1) else (k + 1)

def eca_kernel_size(channels: int, gamma: int = 2, b: int = 1, k_min: int = 1, k_max: int = 15) -> int:
    k = int(abs((math.log2(max(1, channels)) / gamma) + b))
    k = _make_odd(max(k_min, min(k, k_max)))
    return k

class ECABlock(nn.Module):
    def __init__(self, channels: int, gamma: int = 2, b: int = 1):
        super().__init__()
        k = eca_kernel_size(channels, gamma=gamma, b=b)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.conv1d = nn.Conv1d(1, 1, kernel_size=k, padding=(k - 1)//2, bias=False)
        self.gate = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        s = self.pool(x).squeeze(-1).squeeze(-1)   # (B,C)
        s = s.unsqueeze(1)                         # (B,1,C)
        a = self.conv1d(s)                         # (B,1,C)
        w = self.gate(a).squeeze(1).unsqueeze(-1).unsqueeze(-1)  # (B,C,1,1)
        return x * w

# 3) ECA’yı bir Conv bloğun içine takalım

ECA tek başına “model” değil; blok içinde kullanılır.

In [4]:
class ConvBNAct(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False),
            nn.BatchNorm2d(cout),
            nn.SiLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)

class ConvECA(nn.Module):
    def __init__(self, cin, cout, stride=1):
        super().__init__()
        self.conv = ConvBNAct(cin, cout, k=3, s=stride, p=1)
        self.eca = ECABlock(cout)

    def forward(self, x):
        x = self.conv(x)
        x = self.eca(x)
        return x


In [5]:
x = torch.randn(2, 64, 56, 56)
blk = ConvECA(64, 128, stride=1)
y = blk(x)
print(y.shape)


torch.Size([2, 128, 56, 56])


# 4) Basit bir ECA’lı classification modeli yazalım

In [6]:
class ECANetLite(nn.Module):
    def __init__(self, num_classes=10, in_ch=3, base=32):
        super().__init__()
        self.stem = nn.Sequential(
            ConvBNAct(in_ch, base, k=3, s=2, p=1),
            ECABlock(base),
        )

        self.stage1 = nn.Sequential(
            ConvECA(base, base, stride=1),
            ConvECA(base, base, stride=1),
        )
        self.stage2 = nn.Sequential(
            ConvECA(base, base*2, stride=2),
            ConvECA(base*2, base*2, stride=1),
        )
        self.stage3 = nn.Sequential(
            ConvECA(base*2, base*4, stride=2),
            ConvECA(base*4, base*4, stride=1),
        )

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(base*4, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.pool(x).flatten(1)
        return self.fc(x)


In [7]:
m = ECANetLite(num_classes=10, in_ch=3, base=32)
x = torch.randn(2, 3, 224, 224)
y = m(x)
print(y.shape)


torch.Size([2, 10])


# ECA’lı Residual Block

In [8]:
import math
import torch
import torch.nn as nn

def _make_odd(k: int) -> int:
    return k if (k % 2 == 1) else (k + 1)

def eca_kernel_size(channels: int, gamma: int = 2, b: int = 1, k_min: int = 1, k_max: int = 15) -> int:
    k = int(abs((math.log2(max(1, channels)) / gamma) + b))
    k = _make_odd(max(k_min, min(k, k_max)))
    return k

class ECABlock(nn.Module):
    def __init__(self, channels: int, gamma: int = 2, b: int = 1):
        super().__init__()
        k = eca_kernel_size(channels, gamma=gamma, b=b)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.conv1d = nn.Conv1d(1, 1, kernel_size=k, padding=(k - 1)//2, bias=False)
        self.gate = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        s = self.pool(x).squeeze(-1).squeeze(-1).unsqueeze(1)
        a = self.conv1d(s)
        w = self.gate(a).squeeze(1).unsqueeze(-1).unsqueeze(-1)
        return x * w

class ConvBNAct(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1, act="silu"):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        self.act = nn.SiLU(inplace=True) if act == "silu" else nn.ReLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class ECAResBlock(nn.Module):
    def __init__(self, cin, cout, stride=1, gamma=2, b=1):
        super().__init__()
        self.conv1 = ConvBNAct(cin, cout, k=3, s=stride, p=1, act="silu")
        self.conv2 = nn.Sequential(
            nn.Conv2d(cout, cout, 3, padding=1, bias=False),
            nn.BatchNorm2d(cout),
        )
        self.eca = ECABlock(cout, gamma=gamma, b=b)
        self.out_act = nn.SiLU(inplace=True)

        self.short = None
        if stride != 1 or cin != cout:
            self.short = nn.Sequential(
                nn.Conv2d(cin, cout, 1, stride=stride, bias=False),
                nn.BatchNorm2d(cout)
            )

    def forward(self, x):
        identity = x if self.short is None else self.short(x)
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.eca(y)
        y = y + identity
        return self.out_act(y)

class ECAResNetLite(nn.Module):
    def __init__(self, num_classes=10, in_ch=3, base=32, gamma=2, b=1):
        super().__init__()
        self.stem = nn.Sequential(
            ConvBNAct(in_ch, base, k=3, s=2, p=1),
            ConvBNAct(base, base, k=3, s=1, p=1),
        )

        self.stage1 = nn.Sequential(
            ECAResBlock(base, base, stride=1, gamma=gamma, b=b),
            ECAResBlock(base, base, stride=1, gamma=gamma, b=b),
        )
        self.stage2 = nn.Sequential(
            ECAResBlock(base, base*2, stride=2, gamma=gamma, b=b),
            ECAResBlock(base*2, base*2, stride=1, gamma=gamma, b=b),
        )
        self.stage3 = nn.Sequential(
            ECAResBlock(base*2, base*4, stride=2, gamma=gamma, b=b),
            ECAResBlock(base*4, base*4, stride=1, gamma=gamma, b=b),
        )

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(base*4, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.pool(x).flatten(1)
        return self.fc(x)

if __name__ == "__main__":
    m = ECAResNetLite(num_classes=10, in_ch=3, base=32)
    x = torch.randn(2, 3, 224, 224)
    y = m(x)
    print(y.shape)


torch.Size([2, 10])
